In [5]:
import gzip
import numpy
from collections import defaultdict
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
def readCSV(path):
    f = gzip.open(path, 'rt') 
    f.readline()
    for l in f:
        yield l.strip().split(',') 
allRatings = []
userBookRatings = defaultdict(list) 
userBooks = defaultdict(list) 
userRatings = defaultdict(list) 
bookRatings = defaultdict(list) 
userBookRatingsValidSet = [] 
userBookRatingsTrainSet = [] 
books = set()
users = set() 
total_set = [] 
training_set = [] 
valid_set = [] 
total_set_r = [] 
c=0
for user,book,r in readCSV("/Users/apple/Desktop/CSE258/HW3/assignment1/train_Interactions.csv.gz"): 
    r = int(r)
    books.add(book)
    users.add(user) 
    total_set.append([user, book, 1]) 
    total_set_r.append([user, book, r])
    
training_set_r = [x for x in total_set_r[:190000]] 
valid_set_r = [x for x in total_set_r[190000:]] 
for item in training_set_r:
    user = item[0] 
    book = item[1] 
    r = int(item[2])
allRatings.append(r)
userRatings[user].append(r) 
bookRatings[book].append(r) 
userBookRatings[user].append([book, r]) 
userBookRatingsTrainSet.append([user, book, r])
for item in valid_set_r:
    user = item[0]
    book = item[1]
    r = int(item[2]) 
    userBookRatingsValidSet.append([user, book, r])
training_set = [x for x in total_set[:190000]] 
valid_set = [x for x in total_set[190000:]] 
bookUsers = defaultdict(set)
for item in training_set:
    user = item[0]
    book = item[1]
    r = int(item[2]) 
    userBooks[user].append(book) 
    bookUsers[book].add(user)

import random 
negative_valid_set = [] 
for d in valid_set:
    u = d[0]
    b = random.choice(tuple(books)) 
    while b in userBooks[u]:
        b = random.choice(tuple(books)) 
    negative_valid_set.append([u, b, 0])
valid_set = valid_set + negative_valid_set 
random.shuffle(valid_set)

bookCount = defaultdict(int) 
totalRead = 0
for user,book,_ in readCSV("/Users/apple/Desktop/CSE258/HW3/assignment1/train_Interactions.csv.gz"): 
    bookCount[book] += 1
    totalRead += 1
mostPopular = [(bookCount[x], x) for x in bookCount] 
mostPopular.sort()
mostPopular.reverse()
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/1.5: break
predictions = [] 
for l in valid_set:
    u,b = l[0], l[1] 
    if b in return1:
        predictions.append(1) 
    else:
        predictions.append(0) 
accuracy = []
for i in range(len(predictions)):
    if predictions[i] == valid_set[i][2]:
        accuracy.append(1) 
    else:
        accuracy.append(0)



In [6]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2)) 
    denom = len(s1.union(s2)) 
    return float(numer) / denom
predictionsJaccard = [] 
thresholdJaccard = 0.008 
for l in valid_set:
    u,b = l[0], l[1] 
    similar = False
    for bPrime in userBooks[u]:
        j = Jaccard(bookUsers[b], bookUsers[bPrime]) 
        if j >= thresholdJaccard:
            similar = True
            break 
    if similar:
        predictionsJaccard.append(1) 
    else:
        predictionsJaccard.append(0) 
accuracyJaccard = []
for i in range(len(predictionsJaccard)):
    if predictionsJaccard[i] == valid_set[i][2]: 
        accuracyJaccard.append(1)
    else: 
        accuracyJaccard.append(0)
print(float(sum(accuracyJaccard))/len(accuracyJaccard))



0.6164


In [9]:
predictionsJaccardPop = [] 
thresholdJaccardPop = 0.00833
return2 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return2.add(i)
    if count > totalRead/1.301: break
for l in valid_set:
    u,b = l[0], l[1]
    similar = False
    for bPrime in userBooks[u]:
        j = Jaccard(bookUsers[b], bookUsers[bPrime])
        if j >= thresholdJaccardPop:
            similar = True
            break
    if similar and b in return2:
        predictionsJaccardPop.append(1) 
    else:
        predictionsJaccardPop.append(0) 
accuracyJaccardPop = []
for i in range(len(predictionsJaccardPop)):
    if predictionsJaccardPop[i] == valid_set[i][2]: 
        accuracyJaccardPop.append(1)
    else: 
        accuracyJaccardPop.append(0)
print(len(accuracyJaccardPop)) 
print(float(sum(accuracyJaccardPop))/len(accuracyJaccardPop))

20000
0.6664


In [10]:
p = open('predictions_Read.txt', 'w') 
for l in open("/Users/apple/Desktop/CSE258/HW3/assignment1/pairs_Read.txt"): 
    if l.startswith("userID"):
        p.write(l)
        continue
    u,b = l.strip().split('-') 
    similar = False
    for bPrime in userBooks[u]:
        j = Jaccard(bookUsers[b], bookUsers[bPrime]) 
        if j >= thresholdJaccardPop:
            similar = True
            break
    if similar and b in return2:
        s = str(u) + '-' + str(b)+',1\n'
        p.write(s)
    else:
        s = str(u) + '-' + str(b)+',0\n'
        p.write(s)
p.close()